<a href="https://colab.research.google.com/github/abhijit-z/IDS_506_Omicron_Tweets_Analysis/blob/main/Classification_using_demographer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Classifying the tweets based on the source (individual vs organization). Using the demographer package the tweets have been classified into individual and organizations

In [ ]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import os

import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
!ls

drive  sample_data


In [ ]:
#%cd "/content/drive/MyDrive/Omicron Tweet Analytics project/Initial_dataset"

In [ ]:
#!ls

In [ ]:
#!ls /usr/local/lib/python3.7/dist-packages/demographer/

In [ ]:
#import tensorflow as tf
#print(tf. __version__)

In [ ]:
#!pip uninstall tensorflow 
#!pip install  tensorflow==1.15
#!pip install tensorflow==1.13.1

In [ ]:
!pip install demographer


import json
import pandas as pd
import numpy as np
from demographer import process_tweet
#from google.colab import drive
from scipy.stats import mode


'''
# Commented out IPython magic to ensure Python compatibility.
!ls
# %cd /

# Commented out IPython magic to ensure Python compatibility.
# %cd /
# %cd content/drive/MyDrive
'''

from demographer.indorg_neural import NeuralOrganizationDemographer
from demographer.indorg import IndividualOrgDemographer


In [ ]:
demographer_list = [
        IndividualOrgDemographer(setup='balanced'),
        NeuralOrganizationDemographer(setup='balanced'),
        IndividualOrgDemographer(setup='full'),
        NeuralOrganizationDemographer(setup='full')
        
    ]

required_cols = ['friends_count', 'followers_count', 'listed_count', 'statuses_count', 'description', 'verified', 'created_at', 'name', 'screen_name']

cols_in_hydrated = []

for col in required_cols:
  cols_in_hydrated.append('user_' + col)

date1 = '2022-01-11'
date2 = '2022-01-31'
#date2 = '2022-01-31'

#date1 = datetime.datetime.strptime('12012021', '%m%d%Y')
#date2 = datetime.datetime.strptime('12022021', '%m%d%Y')

mydates = pd.date_range(date1, date2, freq='d')#.tolist()
print(mydates)
for d in pd.date_range(date1, date2, freq='d'):
  d_str = d.strftime("%m%d%Y")
  print(d_str)
  path = "/content/drive/MyDrive/Omicron Tweet Analytics project/Initial_dataset/tweet_" + d_str + ".csv"

  df = pd.read_csv(path, engine='python', on_bad_lines = 'skip')
  df_sub = df[cols_in_hydrated]
  df_sub.columns = required_cols

  for col in df_sub.columns:
    if df_sub[col].dtype=='object':
      df_sub[col] = df_sub[col].fillna('')
    else:
      df_sub[col] = df_sub[col].fillna(0)
      
  def ind_org_classify(tweet):
    result = process_tweet(tweet, demographer_list)
    demo_list = ['indorg_balanced', 'indorg_full', 'indorg_neural_balanced', 'indorg_neural_full']
    output = []
    for demo in demo_list:
      output.append(result.get(demo).get('value'))
    return mode(output)[0][0]

  # result.get('indorg_balanced').get('value')
  tweet_list = df_sub.to_dict(orient='records')
  # tweet_list = df_sub.head(2000).to_dict(orient='records')
  #result = map(ind_org_classify, tweet_list)

  #df_sub.head(2000)['ind_org'] = list(result)

  df_new = df

  df_new['ind_org'] = list(map(ind_org_classify, tweet_list))

  tweet_list
  print(df_new.shape)
  df_new[cols_in_hydrated + ['ind_org']]

  print(df_new['ind_org'].value_counts())

  df_ind = df_new[df_new['ind_org']== 'ind']

  path2 = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter1_res/IndivFiles"

  #os.makedirs(path, exist_ok=False)
  isExist = os.path.exists(path2)
  if not isExist:
  
    # Create a new directory because it does not exist 
    os.makedirs(path2)
    print("The new directory is created!")
  dest_path = "/content/drive/MyDrive/Omicron Tweet Analytics project/Filter1_res/IndivFiles/df_ind_" + d_str + ".csv"
  df_ind.to_csv(dest_path)






INFO:tensorflow:Restoring parameters from /usr/local/lib/python3.7/dist-packages/demographer/models/indorg_neural/balanced
INFO:tensorflow:Restoring parameters from /usr/local/lib/python3.7/dist-packages/demographer/models/indorg_neural/full
DatetimeIndex(['2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14',
               '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18',
               '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22',
               '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26',
               '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30',
               '2022-01-31'],
              dtype='datetime64[ns]', freq='D')
01112022


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(135850, 36)
ind    104522
org     31328
Name: ind_org, dtype: int64
01122022
(132016, 36)
ind    102255
org     29761
Name: ind_org, dtype: int64
01132022
(135718, 36)
ind    103714
org     32004
Name: ind_org, dtype: int64
01142022
(115288, 36)
ind    87334
org    27954
Name: ind_org, dtype: int64
01152022
(80550, 36)
ind    65137
org    15413
Name: ind_org, dtype: int64
01162022
(72848, 36)
ind    58539
org    14309
Name: ind_org, dtype: int64
01172022
(88112, 36)
ind    66190
org    21922
Name: ind_org, dtype: int64
01182022
(111752, 36)
ind    83559
org    28193
Name: ind_org, dtype: int64
01192022
(111774, 36)
ind    83282
org    28492
Name: ind_org, dtype: int64
01202022
(86183, 36)
ind    64189
org    21994
Name: ind_org, dtype: int64
01212022
(92204, 36)
ind    68382
org    23822
Name: ind_org, dtype: int64
01222022
(69696, 36)
ind    56122
org    13574
Name: ind_org, dtype: int64
01232022
(65240, 36)
ind    52341
org    12899
Name: ind_org, dtype: int64
01242022
(93780, 36)
i

In [ ]:
#!pip install  tensorflow==1.15

In [ ]:
#!pip uninstall tensorflow==2.8.0

In [ ]:
import tensorflow as tf
print(tf. __version__)

1.15.0


In [ ]:
%cd "/usr/local/lib/python3.7/dist-packages/demographer/models"

/usr/local/lib/python3.7/dist-packages/demographer/models


In [ ]:
!ls

indorg_neural	      indorg_simple	    placeholder.txt
indorg_neural.tar.gz  indorg_simple.tar.gz


In [ ]:
df_new['ind_org'].value_counts()

ind    70777
org    22697
Name: ind_org, dtype: int64

## Ignore below this